In [1]:
import os
DATASET_NAME = "test_project-3cocv-pfiyk"
DATASET_ROOT_FOLDER = os.path.join("data",DATASET_NAME)
MODEL_NAME = "sportimization-1b"
MODEL_ARCH = "v9-c"
NUM_EPOCH = 1
BATCH_SIZE = 1
IMAGE_SIZE = 640
RESUME = ""
DATASET_VERSION = "coco" # yolov7,yolov9,yolov11 , only coco is working correctly?
OUTPUT_PATH = "runs"#"/content/drive/MyDrive/"+MODEL_NAME #runs

In [6]:
import roboflow


roboflow.login()

rf = roboflow.Roboflow()
project = rf.workspace("ws-tmip7").project(DATASET_NAME)
dataset = project.version(1).download(model_format="coco",location=os.path.join(".","data",DATASET_NAME))


You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./data/test_project-3cocv-pfiyk in coco:: 100%|██████████| 12626/12626 [00:03<00:00, 3734.65it/s]


In [5]:
# directory structure bullshit....
# convert roboflow directory structure and filenames to YOLO MIT version

import shutil
import glob
import json

image_dir = os.path.join(DATASET_ROOT_FOLDER,"images")
labels_dir = os.path.join(DATASET_ROOT_FOLDER,"labels")
annotations_dir =os.path.join(DATASET_ROOT_FOLDER,"annotations")
phases = ["train","valid","test"]
for phase in phases:
    os.makedirs(os.path.join(image_dir,phase),exist_ok=True)
    imgs_to_move = glob.glob(os.path.join(DATASET_ROOT_FOLDER,phase,"**","*.jpg"),recursive=True)
    #Yolo MIT needs them in dataset_root/images/<phase>/
    for img in imgs_to_move:
        shutil.move(img,os.path.join(image_dir,phase))

    #labels for yolo datasets
    #Yolo MIT needs them in datsset_root/labels/<phase>/
    if os.path.isdir(os.path.join(DATASET_ROOT_FOLDER,phase,"labels")):
        os.makedirs(os.path.join(labels_dir,phase),exist_ok=True)
        labels_to_move = glob.glob(os.path.join(DATASET_ROOT_FOLDER,phase,"labels","*.txt"))
        for label in labels_to_move:
            shutil.move(label,os.path.join(labels_dir,phase))

    #annotations for coco datasets. roboflow puts them in the image directory
    #yolo MIT needs them in dataset_root/annotations/instances_<phase>.json
    annotations_to_move = glob.glob(os.path.join(DATASET_ROOT_FOLDER,phase,"**","*.json"),recursive=True)
    if len(annotations_to_move) >0:
        if(len(annotations_to_move)>1):
            raise Exception("Expected one annotation json")
        os.makedirs(annotations_dir,exist_ok=True)
        for annotation_file in annotations_to_move:
            annotation_data = dict()
            #roboflow creates a dummy category with id 0...no idea why....
            with open(annotation_file,"r") as json_file:
                annotation_data = json.load(json_file)
                annotation_data["categories"].pop(0)
                for category in annotation_data["categories"]:
                    category['supercategory'] = 'football'
            with open(os.path.join(annotations_dir,"instances_"+phase+".json","w")) as json_file:
                json.dump(annotation_data,json_file)




[{'id': 1, 'name': 'ball', 'supercategory': 'football'}, {'id': 2, 'name': 'goalkeeper', 'supercategory': 'football'}, {'id': 3, 'name': 'player', 'supercategory': 'football'}, {'id': 4, 'name': 'referee', 'supercategory': 'football'}]
[{'id': 1, 'name': 'ball', 'supercategory': 'football'}, {'id': 2, 'name': 'goalkeeper', 'supercategory': 'football'}, {'id': 3, 'name': 'player', 'supercategory': 'football'}, {'id': 4, 'name': 'referee', 'supercategory': 'football'}]
[{'id': 1, 'name': 'ball', 'supercategory': 'football'}, {'id': 2, 'name': 'goalkeeper', 'supercategory': 'football'}, {'id': 3, 'name': 'player', 'supercategory': 'football'}, {'id': 4, 'name': 'referee', 'supercategory': 'football'}]


In [17]:
# modify yolo config file
import yaml



with open("yolo/config/config.yaml","r") as file:
    data = yaml.safe_load(file)
    data["name"] = MODEL_NAME
    data["resume"] = RESUME
    defaults = data["defaults"]
    for item in defaults:
        if isinstance(item, dict):
            if "model" in item.keys():
                item["model"] = MODEL_ARCH
            elif "dataset" in item.keys():
                item["dataset"] = DATASET_NAME
            elif "task" in item.keys():
                item["task"] = "train-"+MODEL_NAME
    defaults[-1] = "general-"+MODEL_NAME
    hydra = data["hydra"]
    run = hydra["run"]
    run["dir"] = OUTPUT_PATH


        
    print(data)    
    with open(os.path.join("yolo","config",MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open("yolo/config/general.yaml","r") as file:
    data = yaml.safe_load(file)
    data["use_wandb"] = False
    data["use_tensorboard"] = False
    #data["cpu_num"] = 8
    data["image_size"] = [IMAGE_SIZE,IMAGE_SIZE]
    data["out_path"] = OUTPUT_PATH
    with open(os.path.join("yolo","config","general-"+MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open("yolo/config/task/train.yaml","r") as file:
    data = yaml.safe_load(file)
    data["epoch"] = NUM_EPOCH
    d = data["data"]
    d["batch_size"]= BATCH_SIZE
    with open(os.path.join("yolo","config","task","train-"+MODEL_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)

with open(os.path.join("data",DATASET_NAME,"data.yaml"),"r") as file:
    data = yaml.safe_load(file)
    data["path"] = os.path.join("data",DATASET_NAME)
    data["train"] = "train"
    data["validation"] = "valid"
    data["test"] = "test"
    data["class_num"] = data["nc"]
    data.pop("nc")
    data.pop("val")
    with open(os.path.join("yolo","config","dataset",DATASET_NAME+".yaml"),"w") as out:
        yaml.safe_dump(data=data,stream=out,sort_keys=False)


{'hydra': {'run': {'dir': 'runs'}}, 'name': 'sportimization-1', 'resume': True, 'defaults': ['_self_', {'task': 'train-sportimization-1'}, {'dataset': 'football-players-detection-3zvbc'}, {'model': 'v9-s'}, 'general-sportimization-1']}


FileNotFoundError: [Errno 2] No such file or directory: 'data/football-players-detection-3zvbc/data.yaml'

In [ ]:
# start training
